In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 101 kB 8.4 MB/s 
     |████████████████████████████████| 6.6 MB 50.1 MB/s 
     |████████████████████████████████| 596 kB 38.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 157 kB 47.3 MB/s 
     |████████████████████████████████| 181 kB 52.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 157 kB 51.5 MB/s 
     |████████████████████████████████| 157 kB 58.6 MB/s 
     |████████████████████████████████| 156 kB 55.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=c565d2ca59b9dbfd40e253095e6b6ce8c6b61aad1ff918265edd32d714da8c72
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [3]:
import re
import os
import math
from timeit import default_timer as timer
from typing import Iterable, List
from IPython.display import Image
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch import Tensor
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchsummary import summary

from torchtext.data.utils import get_tokenizer
from torchtext.data.functional import to_map_style_dataset
from torchtext.vocab import build_vocab_from_iterator
import argparse

import torch.nn.functional as F

from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

from tqdm import tqdm

In [4]:
import wandb

wandb.init(project="my-project", entity="whatsbird")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
import easydict
 
args = easydict.EasyDict({
        "batch": 64,
        "epoch": 5,
        "device": "cuda",
        "learning_rate": 2e-5
})

# data loader

In [6]:
os.mkdir('data')

In [7]:
train_data = datasets.MNIST(root='data', train=True, download=True,transform=ToTensor())
test_data = datasets.MNIST(root='data', train=False, download=True,transform=ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



# 🤓  Knowledge Distillation

- model compression의 방법 중 하나임. 
- 잘 학습시킨 거대한 teacher model로부터 작게 만든 student model을 학습시키는 것임. 
- teacher model의 soft label(probability)와 student model의 prediction을 함께 이용해서 loss function을 최소화시키는 방향으로 학습함. 
  - soft label : temperature scaling function을 softmax에 적용해줌. probability distribution을 스무딩해주고 inter-class relationship에 대해서도 학습할 수 있음.

# 🥑 Distiller() class를 만들어 봅시다
- a trained teacher model
- a student model to train
- student loss function : student prediction과 label사이의 차이
- distillation loss function : soft student predictions와 soft teacher labels사이의 차이
  - temperature scaling
- alpha factor : student loss 와 distillation loss의 weight임.
  - alpha * student loss + (1-alpha) * distillation loss
- optimizer




In [8]:
#gpu사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# 🥑  Create student and teacher models

In [9]:
a = torch.randn((64,1,28,28))
m = nn.Conv2d(1,256,kernel_size=(3,3),stride=(2,2))
output = m(a)
print(output.size())
m = nn.MaxPool2d(2,1)
output = m(output)
print(output.size())
m = nn.Conv2d(256,512,kernel_size=(3,3),stride=(2,2)) 
output = m(output)
print(output.size())
m = nn.Flatten()
output = m(output)
print(output.size())

torch.Size([64, 256, 13, 13])
torch.Size([64, 256, 12, 12])
torch.Size([64, 512, 5, 5])
torch.Size([64, 12800])


In [10]:
class Teacher(nn.Module):
  def __init__(self):
    super(Teacher,self).__init__()
    self.conv1 = nn.Conv2d(1,256,kernel_size=(3,3),stride=(2,2))  
    self.pool = nn.MaxPool2d(2,1)
    self.conv2 = nn.Conv2d(256,512,kernel_size=(3,3),stride=(2,2))
    self.linear = nn.Linear(512*5*5, 10)
  def forward(self,x):
    conv1 = F.leaky_relu(self.conv1(x),0.2) #([64, 256, 13, 13])
    pool = self.pool(conv1) #[64, 256, 12, 12]
    conv2 = self.conv2(pool) #[64, 512, 5, 5]
    flatten = torch.flatten(conv2,1) # flatten all dimensions except batch
    output = self.linear(flatten)
    return output

class Student(nn.Module):
  def __init__(self):
    super(Student,self).__init__()
    self.conv1 = nn.Conv2d(1,16,kernel_size=(3,3),stride=(2,2))  
    self.pool = nn.MaxPool2d(2,1) 
    self.conv2 = nn.Conv2d(16,32,kernel_size=(3,3),stride=(2,2))
    self.linear = nn.Linear(32*5*5, 10)
  def forward(self,x):
    conv1 = F.leaky_relu(self.conv1(x),0.2) 
    pool = self.pool(conv1) 
    conv2 = self.conv2(pool) 
    flatten = torch.flatten(conv2,1) 
    output = self.linear(flatten)
    return output

teacher = Teacher().to(args.device)
student = Student().to(args.device)
student_scratch = Student().to(args.device)

In [11]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
distillation_criterion = nn.KLDivLoss(reduction="batchmean")

teacher_optimizer = optim.Adam(teacher.parameters(), lr=0.001)
student_optimizer = optim.Adam(student.parameters(), lr=0.001)
student_scratch_optimizer = optim.Adam(student_scratch.parameters(), lr=0.001)

In [12]:
def test(model, data_loader, criterion,device=args.device):
  correct = 0
  total = 0
  test_loss = 0
  # since we're not training, we don't need to calculate the gradients for our outputs
  model.eval() #train과 inference 구분을 위함. examples) Dropouts Layers, BatchNorm Layers etc. You need to turn off them during model evaluation, and .eval() will do it for you
  with torch.no_grad():
      for step, (inputs, labels) in enumerate(tqdm(data_loader)):
          inputs, labels = inputs.to(device), labels.to(device)
          # predict
          outputs = model(inputs)

          # loss
          loss = criterion(outputs, labels)
          test_loss += loss.item()

          # total loss and acc
          predicted = outputs.argmax(dim=1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  
  avg_test_loss = test_loss / len(data_loader)
  avg_train_accuracy = 100 * correct / total
  print("Test : Loss : {0:.4f} | Acc : {1:.4f} | {2}/{3}".format(avg_test_loss, avg_train_accuracy,correct,total))

In [13]:
def train(model,data_loader,criterion,optimizer,device=args.device,epochs=5,
          teacher=None,distillation_criterion=None, alpha=0.1, temperature=3):
  print('start training')
  for epoch in range(epochs):
    print(f'Epoch: {epoch+1}/{args.epoch}')
    train_loss =0.0
    distill_loss = 0.0
    correct = 0
    total = 0
    for step, (inputs, labels) in enumerate(tqdm(data_loader)):
      # batch is a list of [inputs, labels]
      inputs, labels = inputs.to(device), labels.to(device)
      #zero the parameter gradients
      optimizer.zero_grad()

      #forward
      outputs = model(inputs)

      #distillation
      if teacher != None:
        teacher_outputs = teacher(inputs)
      
      #backward
      loss = criterion(outputs, labels)
      if distillation_criterion != None:
        student_outputs = F.log_softmax(outputs/temperature)
        teacher_outputs = F.softmax(teacher_outputs/temperature)

        distillation_loss = distillation_criterion(student_outputs, teacher_outputs) # input should be a distribution in the log space
        distill_loss += distillation_loss.item()
        loss = alpha * loss + (1-alpha) * distillation_loss 
      loss.backward()

      #optimize
      optimizer.step()

      #for evaluation
      train_loss += loss.item()
      # total loss and acc
      predicted = outputs.argmax(dim=1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
    
    print("Epoch {0}/{1} : Loss : {2:.4f} | Acc : {3:.4f} | {4}/{5}"\
              .format(epoch+1, args.epoch, train_loss / len(data_loader), 100 * correct / total, correct, total))
  print()
  return model

# 🥑 Prepare the dataset

In [14]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data , batch_size=args.batch, shuffle=True)
test_loader = DataLoader(test_data, batch_size=args.batch, shuffle=True)

In [15]:
image, label = next(iter(train_loader))
print(image.shape)

torch.Size([64, 1, 28, 28])


# 🥑  Train the teacher

In [16]:
teacher = train(teacher,train_loader,criterion,teacher_optimizer)
test(teacher, test_loader, criterion,device=args.device)

start training
Epoch: 1/5


100%|██████████| 938/938 [00:23<00:00, 39.69it/s]


Epoch 1/5 : Loss : 0.1713 | Acc : 94.8750 | 56925/60000
Epoch: 2/5


100%|██████████| 938/938 [00:15<00:00, 59.42it/s]


Epoch 2/5 : Loss : 0.0870 | Acc : 97.4150 | 58449/60000
Epoch: 3/5


...

Epoch 4/5 : Loss : 0.0608 | Acc : 98.2017 | 58921/60000
Epoch: 5/5


100%|██████████| 938/938 [00:15<00:00, 59.06it/s]


Epoch 5/5 : Loss : 0.0545 | Acc : 98.3633 | 59018/60000



100%|██████████| 157/157 [00:01<00:00, 84.86it/s]

Test : Loss : 0.0861 | Acc : 97.7600 | 9776/10000


# 🥑  Distill teacher to student

In [17]:
student = train(student,train_loader,criterion,student_optimizer,device=args.device,epochs=5,
          teacher=teacher,distillation_criterion=distillation_criterion, alpha=0.1, temperature=10)
test(student, test_loader, criterion,device=args.device)

start training
Epoch: 1/5


  0%|          | 0/938 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
100%|██████████| 938/938 [00:17<00:00, 53.49it/s]


Epoch 1/5 : Loss : 0.1597 | Acc : 89.6633 | 53798/60000
Epoch: 2/5


100%|██████████| 938/938 [00:17<00:00, 53.38it/s]


Epoch 2/5 : Loss : 0.0474 | Acc : 96.5900 | 57954/60000
Epoch: 3/5


...

Epoch 4/5 : Loss : 0.0297 | Acc : 97.5883 | 58553/60000
Epoch: 5/5


100%|██████████| 938/938 [00:17<00:00, 53.46it/s]


Epoch 5/5 : Loss : 0.0265 | Acc : 97.8500 | 58710/60000



100%|██████████| 157/157 [00:01<00:00, 92.84it/s]

Test : Loss : 0.0803 | Acc : 97.8400 | 9784/10000


# 🥑  Train student from scratch for comparison

In [18]:
student_scratch = train(student_scratch,train_loader,criterion,student_scratch_optimizer)
test(student_scratch, test_loader, criterion,device=args.device)

start training
Epoch: 1/5


100%|██████████| 938/938 [00:12<00:00, 73.62it/s]


Epoch 1/5 : Loss : 0.3271 | Acc : 90.4500 | 54270/60000
Epoch: 2/5


100%|██████████| 938/938 [00:12<00:00, 73.94it/s]


Epoch 2/5 : Loss : 0.1107 | Acc : 96.7200 | 58032/60000
Epoch: 3/5


...

Epoch 4/5 : Loss : 0.0770 | Acc : 97.6500 | 58590/60000
Epoch: 5/5


100%|██████████| 938/938 [00:12<00:00, 74.36it/s]


Epoch 5/5 : Loss : 0.0697 | Acc : 97.8750 | 58725/60000



100%|██████████| 157/157 [00:01<00:00, 94.08it/s]

Test : Loss : 0.0653 | Acc : 97.9200 | 9792/10000
